In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [3]:
train_dir = 'archive (1)/train/train/'
test_dir = 'archive (1)/test/test/'

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [5]:
test_datagen = ImageDataGenerator(rescale=1./255,
                                 )

In [6]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 18000 images belonging to 20 classes.
Found 6000 images belonging to 20 classes.


In [7]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Increased dropout for regularization
outputs = Dense(20, activation='softmax')(x)  # Assuming 20 classes

model = Model(inputs=base_model.input, outputs=outputs)

In [8]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

In [10]:
history = model.fit(
    train_generator,
    epochs=50,  # Increased epochs
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/50


C:\Users\numan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


563/563 ━━━━━━━━━━━━━━━━━━━━ 371s 647ms/step - accuracy: 0.5502 - loss: 1.4330 - val_accuracy: 0.8067 - val_loss: 0.5234 - learning_rate: 0.0010
Epoch 2/50
563/563 ━━━━━━━━━━━━━━━━━━━━ 316s 557ms/step - accuracy: 0.8873 - loss: 0.3641 - val_accuracy: 0.8750 - val_loss: 0.3505 - learning_rate: 0.0010
Epoch 3/50
563/563 ━━━━━━━━━━━━━━━━━━━━ 311s 549ms/step - accuracy: 0.9147 - loss: 0.2605 - val_accuracy: 0.8888 - val_loss: 0.2861 - learning_rate: 0.0010
Epoch 4/50
563/563 ━━━━━━━━━━━━━━━━━━━━ 315s 556ms/step - accuracy: 0.9328 - loss: 0.2069 - val_accuracy: 0.8767 - val_loss: 0.3134 - learning_rate: 0.0010
Epoch 5/50
563/563 ━━━━━━━━━━━━━━━━━━━━ 318s 560ms/step - accuracy: 0.9379 - loss: 0.1727 - val_accuracy: 0.8892 - val_loss: 0.2702 - learning_rate: 0.0010
Epoch 6/50
563/563 ━━━━━━━━━━━━━━━━━━━━ 321s 566ms/step - accuracy: 0.9419 - loss: 0.1756 - val_accuracy: 0.8607 - val_loss: 0.3675 - learning_rate: 0.0010
Epoch 7/50
563/563 ━━━━━━━━━━━━━━━━━━━━ 321s 566ms/step - accuracy: 0.9480 

In [11]:
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')

188/188 ━━━━━━━━━━━━━━━━━━━━ 44s 233ms/step - accuracy: 0.9148 - loss: 0.2151
Test accuracy: 0.91


In [13]:
model.save('/Users/numan/OneDrive/Desktop/ByteUprise Internship/Task 1 Hand Gesture/model.h5')